## Importing libraries

In [1]:
# Basic Libraries
import json
import pandas as pd
from configparser import ConfigParser
import datetime
config = ConfigParser()
import asyncio
import nest_asyncio
nest_asyncio.apply()

# Telegram module
from Tools import telegramscraping
from Tools import datapreprocessing
from Tools import sentimentscoring

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Setting up Telegram client

In [2]:
# Setting configuration values
config.read('/Users/pauldoan/Documents/Token Metrics/1 Admin/telegram_credentials.ini')

api_id = config['Telegram']['api_id']
api_hash = config['Telegram']['api_hash']
phone = config['Telegram']['phone']
username = config['Telegram']['username']

from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError

# Create the client and connect
client = TelegramClient(username, api_id, api_hash)
await client.start()
print("Client successfully created.")
# Ensure you're authorized
if not await client.is_user_authorized():
    client.send_code_request(phone)
    try:
        client.sign_in(phone, input('Enter the code: '))
    except SessionPasswordNeededError:
        client.sign_in(password=input('Password: '))

Client successfully created.


## Retrieving telegram group id from Token Metrics db

In [3]:
channel = telegramscraping.find_tm_group('Chainlink')

Telegram group found: chainlinkofficial


## Getting channel members


In [4]:
members = asyncio.run(telegramscraping.scrape_telegram_members(channel, client))

Getting members data for telegram group: chainlinkofficial ...
Scraped 2002 users...
Scraped 4003 users...
Scraped 6004 users...
Scraped 8004 users...
Scraped 10000 users...
No more participants to scrape.
Done.



In [5]:
members.head()

,id,first_name,last_name,user,phone,is_bot
0,144145193,Ian,Balina,diaryofamademan,None,False
1,1332508099,Urushai,None,urushal,None,False
2,434324582,Сергей Александрович,None,Markusina,None,False
3,1210349723,Anton,Dzyatkovsk,antondzyatkovsk1,None,False
4,485250297,Dusan,Cajic,None,None,False


## Getting channel messages


In [6]:
messages = asyncio.run(telegramscraping.scrape_telegram_messages(channel, client, limit_msgs=5000))

Getting messages data for telegram group: chainlinkofficial ...
Current Offset ID is: 201884 ; Total Messages: 1000
Current Offset ID is: 199428 ; Total Messages: 2000
Current Offset ID is: 197257 ; Total Messages: 3000
Current Offset ID is: 194915 ; Total Messages: 4000
Current Offset ID is: 192521 ; Total Messages: 5000
Limit reached.
Done.



In [7]:
messages.head()

,id,post_author,date,message
0,204220,None,2020-09-02 20:16:56+00:00,hey
1,204214,None,2020-09-02 20:06:13+00:00,Hi guys i cant download ledger live mobile ( a...
2,204208,None,2020-09-02 19:47:57+00:00,"who are you guys and what protocol, coin or to..."
3,204186,None,2020-09-02 17:01:19+00:00,Chainlink - Official Channel (@chainlink): \n\...
4,204183,None,2020-09-02 16:47:16+00:00,Welcome new members to the Chainlink community...


In [8]:
messages_processed = datapreprocessing.preprocess_telegram(messages)
messages_processed.head()

Detecting url only messages...


100%|██████████| 5000/5000 [00:00<00:00, 296279.05it/s]


Cleaning text...


100%|██████████| 4822/4822 [00:00<00:00, 110771.18it/s]

Remaining data: 92 %

Done.


,id,post_author,date,text,day
0,204220,None,2020-09-02 20:16:56+00:00,hey,2020-09-02
1,204214,None,2020-09-02 20:06:13+00:00,Hi guys i cant download ledger live mobile ( a...,2020-09-02
2,204208,None,2020-09-02 19:47:57+00:00,"who are you guys and what protocol, coin or to...",2020-09-02
3,204186,None,2020-09-02 17:01:19+00:00,Chainlink - Official Channel (chainlink): In...,2020-09-02
4,204183,None,2020-09-02 16:47:16+00:00,Welcome new members to the Chainlink community...,2020-09-02


In [9]:
sentimentscoring.get_sentiment(messages_processed)

Computing sentiment with Textblob...


100%|██████████| 4592/4592 [00:00<00:00, 4897.32it/s]


Computing sentiment with Vader...


100%|██████████| 4592/4592 [00:00<00:00, 7569.61it/s]

Done.


,id,post_author,date,text,day,blob_polarity,blob_sentiment,vader_polarity,vader_sentiment,sentiment_score,sentiment
0,204220,None,2020-09-02 20:16:56+00:00,hey,2020-09-02,0.000000,neutral,0.0000,neutral,0.000000,neutral
1,204214,None,2020-09-02 20:06:13+00:00,Hi guys i cant download ledger live mobile ( a...,2020-09-02,0.136364,positive,-0.1431,negative,-0.003368,negative
2,204208,None,2020-09-02 19:47:57+00:00,"who are you guys and what protocol, coin or to...",2020-09-02,-0.156250,negative,0.4374,positive,0.140575,positive
3,204186,None,2020-09-02 17:01:19+00:00,Chainlink - Official Channel (chainlink): In...,2020-09-02,0.675000,very positive,0.8316,very positive,0.753300,very positive
4,204183,None,2020-09-02 16:47:16+00:00,Welcome new members to the Chainlink community...,2020-09-02,0.307054,positive,0.9441,very positive,0.625577,very positive
...,...,...,...,...,...,...,...,...,...,...,...
4994,192531,None,2020-08-13 19:30:04+00:00,Search for Chainlink Community. They are unoff...,2020-08-13,0.000000,neutral,0.0000,neutral,0.000000,neutral
4995,192530,None,2020-08-13 19:29:40+00:00,Do we have Community Based Group ?,2020-08-13,0.000000,neutral,0.0000,neutral,0.000000,neutral
4996,192529,None,2020-08-13 19:29:26+00:00,Hlw Sir,2020-08-13,0.000000,neutral,0.0000,neutral,0.000000,neutral
4997,192528,None,2020-08-13 19:27:59+00:00,Hi,2020-08-13,0.000000,neutral,0.0000,neutral,0.000000,neutral
